In [ ]:
!pip install gcloud

In [ ]:
!gcloud auth application-default login

In [ ]:
##importando pacotes
import pandas as pd
import numpy as np
import time
import os
import pandas_gbq
from google.cloud import bigquery
import glob
import openpyxl
import csv
import re


In [51]:
client = bigquery.Client(project='repositoriodedadosgpsp')
query = "SELECT ano, tipo_ocupacao,sigla_uf,id_municipio, sum(n_profissionais) as quantidade_vinculos  FROM `repositoriodedadosgpsp.Datalake.CNES_vinculo_saude_tipo_ocupacao_v2` where publico_privado='Público' group by ano, tipo_ocupacao, sigla_uf,id_municipio"
df = client.query(query).to_dataframe()
df

/usr/local/lib/python3.10/dist-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


,ano,tipo_ocupacao,sigla_uf,id_municipio,quantidade_vinculos
0,2008,Técnicos e auxiliares de enfermagem,AP,1600600,225
1,2008,Trabalhadores em serviços de promoção e apoio ...,RR,1400407,28
2,2008,Nutricionistas,AP,1600303,31
3,2008,Agentes da saúde e do meio ambiente,AP,1600600,12
4,2008,Trabalhadores de moldagem de metais e de ligas...,RR,1400100,2
...,...,...,...,...,...
1836885,2024,Vigilantes e guardas de segurança,TO,1707108,7
1836886,2024,Tecnólogos e técnicos em métodos de diagnóstic...,TO,1703909,1
1836887,2024,"Trabalhadores de atenção, defesa e proteção a ...",TO,1718204,1
1836888,2024,"Camareiros, roupeiros e afins",TO,1702109,1


In [52]:
df['tipo_2'] = np.where(df['tipo_ocupacao'].str.contains('Psicólogos e psicanalistas'),'Psicólogos e psicanalistas',df['tipo_ocupacao'])

In [53]:
df = df[(df['tipo_2']=='Psicólogos e psicanalistas') & (df['ano']>2017) & (df['ano'] <= 2023)]
df

,ano,tipo_ocupacao,sigla_uf,id_municipio,quantidade_vinculos,tipo_2
617405,2018,Psicólogos e psicanalistas,AP,1600279,3,Psicólogos e psicanalistas
617411,2018,Psicólogos e psicanalistas,DF,5300108,459,Psicólogos e psicanalistas
617418,2018,Psicólogos e psicanalistas,RR,1400282,3,Psicólogos e psicanalistas
617439,2018,Psicólogos e psicanalistas,AP,1600501,4,Psicólogos e psicanalistas
617477,2018,Psicólogos e psicanalistas,RR,1400472,3,Psicólogos e psicanalistas
...,...,...,...,...,...,...
1786522,2023,Psicólogos e psicanalistas,SP,3513900,4,Psicólogos e psicanalistas
1786548,2023,Psicólogos e psicanalistas,SP,3545209,23,Psicólogos e psicanalistas
1786582,2023,Psicólogos e psicanalistas,SP,3504404,1,Psicólogos e psicanalistas
1786748,2023,Psicólogos e psicanalistas,TO,1707207,1,Psicólogos e psicanalistas


In [54]:
## Chamando a quantidade de população.
## Tabela no BQ equivalente ao CSV: repositoriodedadosgpsp.Datalake.IBGE_populacao_censo_2022
pop = pd.read_csv("POP_CD2022_ATT.csv");
pop

,UF,COD. UF,COD. MUNIC,NOME DO MUNICÍPIO,POP. COLETADA,POP. IMPUTADA,POPULAÇÃO,id_municipio
0,RO,11,15,Alta Floresta D'Oeste,20865,629,21494,1100015
1,RO,11,23,Ariquemes,93006,3827,96833,1100023
2,RO,11,31,Cabixi,5100,251,5351,1100031
3,RO,11,49,Cacoal,84923,1964,86887,1100049
4,RO,11,56,Cerejeiras,15297,593,15890,1100056
...,...,...,...,...,...,...,...,...
5565,GO,52,22005,Vianópolis,14345,611,14956,5222005
5566,GO,52,22054,Vicentinópolis,8592,176,8768,5222054
5567,GO,52,22203,Vila Boa,4168,47,4215,5222203
5568,GO,52,22302,Vila Propício,5594,221,5815,5222302


In [55]:
df.head(20)

,ano,tipo_ocupacao,sigla_uf,id_municipio,quantidade_vinculos,tipo_2
617405,2018,Psicólogos e psicanalistas,AP,1600279,3,Psicólogos e psicanalistas
617411,2018,Psicólogos e psicanalistas,DF,5300108,459,Psicólogos e psicanalistas
617418,2018,Psicólogos e psicanalistas,RR,1400282,3,Psicólogos e psicanalistas
617439,2018,Psicólogos e psicanalistas,AP,1600501,4,Psicólogos e psicanalistas
617477,2018,Psicólogos e psicanalistas,RR,1400472,3,Psicólogos e psicanalistas
617515,2018,Psicólogos e psicanalistas,AC,1200203,13,Psicólogos e psicanalistas
617534,2018,Psicólogos e psicanalistas,AC,1200385,2,Psicólogos e psicanalistas
617552,2018,Psicólogos e psicanalistas,AL,2707107,4,Psicólogos e psicanalistas
617555,2018,Psicólogos e psicanalistas,AL,2703759,1,Psicólogos e psicanalistas
617571,2018,Psicólogos e psicanalistas,AL,2707206,1,Psicólogos e psicanalistas


In [56]:
df['id_municipio'] = pd.to_numeric(df['id_municipio'])

<ipython-input-56-a8331f5d24a6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['id_municipio'] = pd.to_numeric(df['id_municipio'])


In [57]:
x=df.pivot_table(index=['ano','sigla_uf','id_municipio','tipo_2'], values='quantidade_vinculos',aggfunc=np.sum)
y = pd.DataFrame(x)
df = y.reset_index()

<ipython-input-57-d833fc746e54>:1: FutureWarning: The provided callable <function sum at 0x7c88c3716dd0> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  x=df.pivot_table(index=['ano','sigla_uf','id_municipio','tipo_2'], values='quantidade_vinculos',aggfunc=np.sum)


In [58]:
df1= df.merge(pop[['UF','NOME DO MUNICÍPIO','id_municipio','POPULAÇÃO']], left_on='id_municipio',right_on='id_municipio').sort_values('quantidade_vinculos', ascending=False)

In [59]:
df1['taxa'] = df1['quantidade_vinculos']/df1['POPULAÇÃO']*1000

In [60]:
sum(df1[df1['ano']==2023]['POPULAÇÃO'])

201488366

In [61]:
sum(df1[df1['ano']==2023]['quantidade_vinculos'])/sum(df1[df1['ano']==2023]['POPULAÇÃO'])*1000

0.23927436088295043

In [62]:
## validando alguns dados
df1[(df1['ano']==2023) & (df1['UF']=='RJ')].sort_values('taxa', ascending=False).head(30)

,ano,sigla_uf,id_municipio,tipo_2,quantidade_vinculos,UF,NOME DO MUNICÍPIO,POPULAÇÃO,taxa
19149,2023,RJ,3305307,Psicólogos e psicanalistas,14,RJ,São Sebastião do Alto,7750,1.806452
18795,2023,RJ,3301207,Psicólogos e psicanalistas,24,RJ,Carmo,17198,1.395511
18927,2023,RJ,3302809,Psicólogos e psicanalistas,22,RJ,Mendes,17502,1.256999
18981,2023,RJ,3303609,Psicólogos e psicanalistas,52,RJ,Paracambi,41375,1.256798
19071,2023,RJ,3304508,Psicólogos e psicanalistas,11,RJ,Rio das Flores,8954,1.228501
19011,2023,RJ,3303955,Psicólogos e psicanalistas,26,RJ,Pinheiral,24298,1.070047
19119,2023,RJ,3305000,Psicólogos e psicanalistas,36,RJ,São João da Barra,36573,0.984333
18831,2023,RJ,3301801,Psicólogos e psicanalistas,11,RJ,Engenheiro Paulo de Frontin,12242,0.898546
18915,2023,RJ,3302601,Psicólogos e psicanalistas,36,RJ,Mangaratiba,41220,0.873362
18765,2023,RJ,3300936,Psicólogos e psicanalistas,12,RJ,Carapebus,13847,0.866614


In [63]:
df1.columns

Index(['ano', 'sigla_uf', 'id_municipio', 'tipo_2', 'quantidade_vinculos',
       'UF', 'NOME DO MUNICÍPIO', 'POPULAÇÃO', 'taxa'],
      dtype='object')

In [64]:
## Renomear colunas

df = df1[['ano', 'sigla_uf', 'id_municipio', 'tipo_2', 'quantidade_vinculos',
        'NOME DO MUNICÍPIO', 'POPULAÇÃO', 'taxa']].rename(columns={'tipo_2':'tipo_ocupacao','UF':'sigla_uf', 'NOME DO MUNICÍPIO':'nome_municipio','POPULAÇÃO':'populacao_domiciliada' })

In [65]:
df

,ano,sigla_uf,id_municipio,tipo_ocupacao,quantidade_vinculos,nome_municipio,populacao_domiciliada,taxa
28420,2018,SP,3550308,Psicólogos e psicanalistas,2169,São Paulo,11451999,0.189399
28425,2023,SP,3550308,Psicólogos e psicanalistas,2042,São Paulo,11451999,0.178309
28424,2022,SP,3550308,Psicólogos e psicanalistas,2035,São Paulo,11451999,0.177698
28423,2021,SP,3550308,Psicólogos e psicanalistas,1897,São Paulo,11451999,0.165648
28422,2020,SP,3550308,Psicólogos e psicanalistas,1773,São Paulo,11451999,0.15482
...,...,...,...,...,...,...,...,...
16331,2021,PI,2207850,Psicólogos e psicanalistas,1,Pavussu,3628,0.275634
16332,2022,PI,2207850,Psicólogos e psicanalistas,1,Pavussu,3628,0.275634
16333,2023,PI,2207850,Psicólogos e psicanalistas,1,Pavussu,3628,0.275634
16340,2018,PI,2207934,Psicólogos e psicanalistas,1,Pedro Laurentino,2458,0.406835


In [66]:
df= df[['ano', 'sigla_uf', 'nome_municipio','id_municipio','populacao_domiciliada', 'tipo_ocupacao',
       'quantidade_vinculos',
       'taxa']]

In [67]:
df

,ano,sigla_uf,nome_municipio,id_municipio,populacao_domiciliada,tipo_ocupacao,quantidade_vinculos,taxa
28420,2018,SP,São Paulo,3550308,11451999,Psicólogos e psicanalistas,2169,0.189399
28425,2023,SP,São Paulo,3550308,11451999,Psicólogos e psicanalistas,2042,0.178309
28424,2022,SP,São Paulo,3550308,11451999,Psicólogos e psicanalistas,2035,0.177698
28423,2021,SP,São Paulo,3550308,11451999,Psicólogos e psicanalistas,1897,0.165648
28422,2020,SP,São Paulo,3550308,11451999,Psicólogos e psicanalistas,1773,0.15482
...,...,...,...,...,...,...,...,...
16331,2021,PI,Pavussu,2207850,3628,Psicólogos e psicanalistas,1,0.275634
16332,2022,PI,Pavussu,2207850,3628,Psicólogos e psicanalistas,1,0.275634
16333,2023,PI,Pavussu,2207850,3628,Psicólogos e psicanalistas,1,0.275634
16340,2018,PI,Pedro Laurentino,2207934,2458,Psicólogos e psicanalistas,1,0.406835


In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 31120 entries, 28420 to 31119
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ano                    31120 non-null  Int64  
 1   sigla_uf               31120 non-null  object 
 2   nome_municipio         31120 non-null  object 
 3   id_municipio           31120 non-null  int64  
 4   populacao_domiciliada  31120 non-null  int64  
 5   tipo_ocupacao          31120 non-null  object 
 6   quantidade_vinculos    31120 non-null  Int64  
 7   taxa                   31120 non-null  Float64
dtypes: Float64(1), Int64(2), int64(2), object(3)
memory usage: 2.2+ MB


Subindo para o GBQ

In [ ]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano de referencia da informacao'),
 bigquery.SchemaField('sigla_uf','STRING',description='Sigla da Unidade da Federação.'),
 bigquery.SchemaField('nome_municipio','STRING',description='Nome do município da observação'),
 bigquery.SchemaField('id_municipio','FLOAT',description='Identificador do município pelo IBGE'),
 bigquery.SchemaField('populacao_domiciliada','FLOAT',description='População domiciliada (prévia do Censo 2022)'),
 bigquery.SchemaField('tipo_ocupacao','STRING',description='Qual a ocupação daquele vínculo'),
 bigquery.SchemaField('quantidade_vinculos','INTEGER',description='Quantidade de vinculos'),
 bigquery.SchemaField('taxa','FLOAT',description='Taxa de psicólogos e psicanalistas por 1000 habitantes')
 ]

In [ ]:
## Subindo para datalake
client = bigquery.Client(project='repositoriodedadosgpsp')
dataset_ref = client.dataset('perfil_remuneracao')

/usr/local/lib/python3.10/dist-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [69]:
table_ref = dataset_ref.table('CNES_psicologos_mil_habitantes_v3') # nome da tabela no padrão FONTE_algo_intuitivo_dado
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result()

LoadJob<project=repositoriodedadosgpsp, location=US, id=abe20dc6-72c2-410b-9a79-541369e58ec5>